In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np
import time

In [2]:
sc    = SparkContext("local[*]", "MediaVarianza_2")
datos = sc.textFile("../../../datos/data_ok.csv")
# datos es un RDD con 5000 strings (cada uno es una fila del csv)

In [3]:
num_col = len(datos.collect()[0].split(" "))
if (num_col != 211):
    print("ERROR EN EL CÁLCULO DE COLUMNAS")
num_filas = len(datos.collect())
print(num_filas)

5000


In [4]:
def todas_columnas_lento():
    """ Esta es la forma lenta de hacerla, sin vectorización."""
    now = time.time()
    for i in range(0, num_col):
        col = datos.map(lambda x: float(x.split(" ")[i]))
        num = col.count()
        media = col.reduce(lambda x, y: x + y)/num
        varia = col.map(lambda x: ((x - media)**2)/num).reduce(lambda x, y: x + y)
        desvi = varia ** 0.5
        print("{}: ".format(i), end='')
        print("  media: {} || varianza: {} || desv. est.: {}".format(media, varia, desvi))

    then = time.time()
    passed = then - now
    print("Tiempo transcurrido: {} segundos".format(passed)) # ~35 segundos

In [5]:
def formato_fila(fila):
    lista = [float(i) for i in fila.split(" ")]
    return np.array(lista)
num     = datos.count() # 5000
data_ok = datos.map(formato_fila) # data_ok es un RDD con 5000 np.arrays de floats

medias    = data_ok.reduce(lambda x, y: x + y)/num # Array con las medias
varianzas = data_ok.map(lambda x: ((x - medias)**2)/num).reduce(lambda x, y: x + y)
desviests = varianzas ** 0.5

# Para comparar con el ejercicio anterior
print(medias[3])
print(varianzas[3])
print(desviests[3])

1409608.9152
5883755024032.185
2425645.27992701
